# Metadata Module

**This is a literate notebook.**

## Motivation

From [Wikipedia](https://en.wikipedia.org/wiki/Metadata)
> Metadata is "data that provides information about other data". In other words, it is "data about data."

What were the conditions on a particular day?  The crew?  What sort of jib settings did we use?  The finishing position?  What were the shroud settings?  How did they perform?  

I started out by writing a email for each race, trying to including learnings, conditions, results.  I moved to creating a Google doc for each race, easier to edit and update. And then I moved to creating a Jupyter notebook for each race day, easier to include data from the actual race all in one place.

The problems with these approaches:

- Repeated work.  Each email/gdoc/notebook is a vague copy of the previous, updated with new info.  This copy/edit process is annoying.
  - For example, one step is to grab the weather/tides, and just this step takes a while by hand.
- The data is locked in a human readable document, not in a machine readable representation.
  - No easy way to generate a single document (i.e. table of contents that shows all races, dates and times).
- No way to analyze the data in one place.  Where can we look to see trends or issues that are inconsistent?

Philosophically, I like metadata which can be searched and cross-referenced.  Data should be easy to edit and update and view.

The solution is to store all this metadata in a single easy to edit datastructure which can then be analyzed/created/edited/rendered for various needs.

## Overview

Code to process race metadata and associate with race logs.

There are currently 3 sources of hand entered metadata, hopefully fewer soon:
- A file called metadata.yaml (in YAML).  This is the final ultimate source for metadata.
  - YAML is super powerful, allowing you to enter complex structured info.
  - It is also designed to be human readable (unlike a database file, or even a CSV).
  - There is no strong schema, and its a bit disorganized.
- A google spreadsheet (gsheet) that is fed by a [Google Form](https://forms.gle/JENZZdSWKNuoF8icA) (which is easy to use on the ride home from the race).
  - The form determines the schema, which can be changed, but it does enforce some structure.
  - The spreadsheet can be downloaded as a CSV from a URL.
- An older pandas dataframe, `log_info.pd` which is should be deprecated.

And a final source, which is a default and empty metadata record generated when we first upload the file.

In all cases above the **key** is the date (and we therefore assume that there is a single "file" per day).  In practice we may have several races on a single day, though these will be in one file. The YAML file will support the ability to discuss the segments.  The Google form does not.

**How to merge duplicates?**. 

- Multiple rows with same date in the gsheet.
  - Delete by hand?  Take the latest?
- Same date in gsheet and YAML.
  - Note, gsheet row will move to YAML when it first appears.
  - Figure out which is newer.  
    - If YAML is newer, keep it.
    - If gsheet is newer, then keep **both**.  Warn user and ask to edit.

  
  

### References 

- Good reference to start: [YAML tutorial](https://rollout.io/blog/yaml-tutorial-everything-you-need-get-started/)
  - [The official reference](https://yaml.org/) Its written in YAML (which makes it a bit weird).
- Nice Google page on [how to use Google Forms](https://zapier.com/learn/google-sheets/how-to-use-google-forms/)


## TODO

- Add timestamp to all YAML entries?  

- Tides? 

- Currents?  

- Weather: wind, etc.  Weather buoy?

- Create a page before the race? 

- Phone images captured during the race 
  - pull them in, link them to the map?
  - extract settings?
  
- Sometimes there are two files from the same day.  In general should not have happened...  but it screws things up.

- Provide a tool to slice a days data into "segments".



## Caveats and concerns

- YAML, as edited by a human author, does not support a strong schema.  Its easy to mess things up, with typos, missing fields, incorrectly named fields, etc.



In [1]:
# imports
import os
import copy
import numbers
import re

import yaml  # We'll use YAML for our metadata
import arrow
import numpy as np
import pandas as pd


# These are libraries written for RegattaAnalysis
from global_variables import G  # global variables
import utils
from utils import DictClass
import process as p
import nbutils
from nbutils import display_markdown, display

In [2]:
# notebook - an example

# Below is snippet of YAML inline.  I am not going to document YAML here.  But notice that the structure
# is reminiscent of Python itself, and it is somewhat readable.

example = """
file: 2020-04-16_14:54.pd.gz
date: "2020-04-16"
title: Tune-up with Creative
purpose: tune_up
conditions: >-
  Beautiful day. Winds were 3 quickly building to 10ish. Flat
  seas. Upwind to Pt. Wells buoy, raised and raced home to the hamburger.
performance: >-
  Good height and speed vs. Creative on the way upwind. Perhaps a bit
  slow at first downwind, exploring to tradeoffs between depth and
  speed.  Best downwind speed when I was at the shrouds and Sara had a
  hand on the mainsheet.
learnings: >-
  Let the sails out for downwind: both main and kite.  Stand forward
  if possible.

  Shroud settings seemed really great, and versatile.  With only 2 on
  the boat, we sailed very well.  These settings are the new base!
raceqs_video: "https://youtu.be/9a5bLeZw8EM"
raceqs: "https://raceqs.com/tv-beta/tv.htm#userId=1146016&divisionId=64190&updatedAt=2020-04-17T18:05:59Z&dt=2020-04-16T15:43:47-07:00..2020-04-16T17:39:12-07:00&boat=Creative"
segments:
  - winds: [6, 12]
    tensions: [29, 10, 0]
    port: [2.251, 1.953, 999]
    stbd: [2.271, 1.959, 999]
    thoughts: >-
      Overall we have had trouble with the Quantum quick tune card,
      where the uppers are a bit too loose or the middles too tight.
      The result is that the mast falls off at the top, rather than
      staying straight or sagging for power.  We took a bit off the
      middle (12 down to 10).
questions:
  - text: Was the prop set correctly?
    author: sara
    context: Were we slower on one tack than the other? 
    proposed_solution: ??
"""

# The data can be trivialy read in this way.  A simple Python datastructure results, in this case
# a dictionary.  
race_metadata = yaml.load(example, Loader=yaml.Loader)

display_markdown("Notice that `segments` and `questions` are sublist of dicts.")
display(race_metadata)

Notice that `segments` and `questions` are sublist of dicts.

{'file': '2020-04-16_14:54.pd.gz',
 'date': '2020-04-16',
 'title': 'Tune-up with Creative',
 'purpose': 'tune_up',
 'conditions': 'Beautiful day. Winds were 3 quickly building to 10ish. Flat seas. Upwind to Pt. Wells buoy, raised and raced home to the hamburger.',
 'performance': 'Good height and speed vs. Creative on the way upwind. Perhaps a bit slow at first downwind, exploring to tradeoffs between depth and speed.  Best downwind speed when I was at the shrouds and Sara had a hand on the mainsheet.',
 'learnings': 'Let the sails out for downwind: both main and kite.  Stand forward if possible.\nShroud settings seemed really great, and versatile.  With only 2 on the boat, we sailed very well.  These settings are the new base!',
 'raceqs_video': 'https://youtu.be/9a5bLeZw8EM',
 'raceqs': 'https://raceqs.com/tv-beta/tv.htm#userId=1146016&divisionId=64190&updatedAt=2020-04-17T18:05:59Z&dt=2020-04-16T15:43:47-07:00..2020-04-16T17:39:12-07:00&boat=Creative',
 'segments': [{'winds': [6, 1

### Render this data to make it more readable

We can prettily easily write code that "renders" these Python structures into readable Markdown.

In [3]:
# code to render a metadata entry in markdown... 

def display_race_summary(race_record):
    "Summarize a race."
    display_markdown(f"# {race_record['title']}: {race_record['date']}")
    rr = race_record.copy()
    for k in "description conditions performance learnings".split():
        if k in rr:
            display_section(k.capitalize(), rr.pop(k))
    links = "raceqs raceqs_video".split()
    if has_key(links, race_record):
        display_markdown("## Links")
        lines = ""
        for k in links:
            if k in rr:
                lines += lines_url(make_title(k), rr.pop(k))
        display_markdown(lines)
    keys = list(rr.keys())
    if len(keys) > 0:
        lines = ""
        display_markdown("## Extras")
        lines = lines_dict("", keys, rr)
        display_markdown(lines)

def has_key(key_list, dictionary):
    for k in key_list:
        if k in dictionary:
            return True
    return False

def is_list_of_dicts(val):
    return isinstance(val, list) and isinstance(val[0], dict)

def lines_dict(prefix, keys, dictionary):
    lines = ""
    for k in keys:
        val = dictionary[k]
        if is_list_of_dicts(val):
            for i, v in enumerate(val):
                lines += f"{prefix}- **{k}: {i}**\n"
                lines += lines_dict(prefix+"  ", v.keys(), v)
        else:
            lines += f"{prefix}- **{k}**: {val}\n"
    return lines
            
def display_section(title, text):
    "Displays a markdown section with text."
    display_markdown(f"## {title}")
    display_markdown(text)        
        
def is_url(s):
    return s.startswith("http")  # Not great, but OK for now

def make_title(key):
    """"
    YAML keys are python keywords (lowercase and separated by underscores).  This converts to a pretty 
    and printable string.
    """
    words = key.split("_")
    words = [w.capitalize() for w in words]
    return " ".join(words)

def lines_url(link_text, url):
    "Displays a markdown URL."
    return f"- [{link_text}]({url})\n"

In [4]:
# notebook - Create a human readable version of this metadata
display_race_summary(race_metadata)

# Tune-up with Creative: 2020-04-16

## Conditions

Beautiful day. Winds were 3 quickly building to 10ish. Flat seas. Upwind to Pt. Wells buoy, raised and raced home to the hamburger.

## Performance

Good height and speed vs. Creative on the way upwind. Perhaps a bit slow at first downwind, exploring to tradeoffs between depth and speed.  Best downwind speed when I was at the shrouds and Sara had a hand on the mainsheet.

...

Let the sails out for downwind: both main and kite.  Stand forward if possible.
Shroud settings seemed really great, and versatile.  With only 2 on the boat, we sailed very well.  These settings are the new base!

## Links

- [Raceqs](https://raceqs.com/tv-beta/tv.htm#userId=1146016&divisionId=64190&updatedAt=2020-04-17T18:05:59Z&dt=2020-04-16T15:43:47-07:00..2020-04-16T17:39:12-07:00&boat=Creative)
- [Raceqs Video](https://youtu.be/9a5bLeZw8EM)


## Extras

- **file**: 2020-04-16_14:54.pd.gz
- **date**: 2020-04-16
- **title**: Tune-up with Creative
- **purpose**: tune_up
- **segments: 0**
  - **winds**: [6, 12]
  - **tensions**: [29, 10, 0]
  - **port**: [2.251, 1.953, 999]
  - **stbd**: [2.271, 1.959, 999]
  - **thoughts**: Overall we have had trouble with the Quantum quick tune card, where the uppers are a bit too loose or the middles too tight. The result is that the mast falls off at the top, rather than staying straight or sagging for power.  We took a bit off the middle (12 down to 10).
- **questions: 0**
  - **text**: Was the prop set correctly?
  - **author**: sara
  - **context**: Were we slower on one tack than the other?
  - **proposed_solution**: ??


In [5]:
# Race metadata is stored as a multi-document sequence in the YAML file.  

def read_metadata():
    """
    Read the race metadata and return a struct, with a 
    - timestamp
    - records:  list of records
    - dates:    dict from date to record
    """
    race_yaml = read_yaml(G.METADATA_PATH)
    dates = {}
    records = []
    for record in race_yaml:
        # If the record is missing a source, assume it was written byhand.
        if 'source' not in record:
            record['source'] = 'byhand'
        dates[record['date']] = record
        records.append(record)
    # File timestamp, used to find valid updates in other sources.
    ts = arrow.get(os.path.getmtime(G.METADATA_PATH)).to('US/Pacific')
    G.logger.info(f"Read {len(records)} records.")
    return DictClass(dates=dates, records=records, timestamp=ts)

def save_metadata(race_records):
    """
    Save a sequence of race records to the metadata file.
    """
    G.logger.info(f"Writing {len(race_records)} records.")    
    utils.backup_file(G.METADATA_PATH)
    # For convenience sort the records by date before writing.
    sorted_records = sorted(race_records, key=lambda r: r['date'])
    with open(G.METADATA_PATH, 'w') as yfs:
        save_yaml(sorted_records, yfs)

# A bit of magic here to ensure we have the best loader/dumper.  Specifying this is required when 
# calling load/dump (below).
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

def read_yaml(yaml_path):
    "Read the race records stored in the YAML file.  Return a dict indexed by date string: YYYY-MM-DD."
    with open(yaml_path, 'r') as yaml_stream:
        race_yaml = list(yaml.load_all(yaml_stream, Loader=Loader))
    return race_yaml

                     
def save_yaml(race_entries, stream=None):
    "Save the race metadata as YAML."
    return yaml.dump_all(race_entries, stream, Dumper=Dumper,
                         default_flow_style=False, sort_keys=False)

In [6]:
# notebook - load some metadata

metadata = read_metadata()
display(metadata.timestamp)

display_race_summary(metadata.dates['2020-04-19'])
display_race_summary(metadata.dates['2020-06-06'])


2020-07-12 23:23:43,486|INFO|read_metadata| Read 70 records.


<Arrow [2020-07-12T11:38:55.526259-07:00]>

# Tune-up with Creative: 2020-04-19

## Description

## Conditions

...

Slower in the up and down but we took a different path

## Learnings

Bilge pump button was in middle again. Maybe our current prediction distracting us?

## Extras

- **file**: 2020-06-06_09:23.pd.gz
- **date**: 2020-06-06
- **title**: Cohab #5
- **begin**: 36271
- **end**: 152106
- **source**: gsheet
- **purpose**: race
- **crew**: ['Sara', 'Paul']
- **wave**: 1-2
- **wind**: [8, 14]
- **port_pointing**: 25-27
- **stbd_pointing**: 25-27
- **settings**: Mostly loose.  Left back stay on too long down wind
- **shroud_name**: Light breeze
- **shroud_tension**: [27, 12, 0]
- **timestamp**: 2020-06-06 22:13:15
- **fluids**: Empty Bilge?
- **shroud_tensions**: 27,12,0


In [7]:
# Since the data for all days is in one place its easy to generate an all up summary.

def summarize(race_records):
    "Summarize each race."
    lines = ""
    for race in race_records:
        lines += f"- **{race['date']}**: {race['title']}\n"
        for key in "description conditions".split():
            if key in race and race[key] is not None and len(race[key]) > 0:
                lines += f"  - *{key.capitalize()}:* {race[key]}\n"
    display_markdown(lines)

def summary_table(race_records, columns = None):
    "Return a summary table the races."
    rows = []
    if columns is None:
        columns = "date title file source".split()
    for race in race_records:
        row = {k:race.get(k, '') for k in columns}
        rows.append(row)
    return pd.DataFrame(rows)


In [8]:
# notebook 

summarize(metadata.records)
summary_table(metadata.records)

- **2019-10-04**: Sail to Everett for Foulweather Bluff Race.
- **2019-10-05**: Foulweather Bluff Race
  - *Description:* Foulweather Bluff Race
- **2019-10-11**: Short practice, upwind tacks and downwind jibes.
- **2019-10-12**: CYC PSSC Day 1
  - *Description:* CYC PSSC Day 1
- **2019-10-18**: Short, at dock.
- **2019-10-19**: STYC Fall Regatta
  - *Description:* STYC Fall Regatta.
- **2019-10-26**: Grand Prix Saturday
  - *Description:* Grand Prix Saturday.
- **2019-11-07**: Short, at dock.
- **2019-11-16**: Snowbird #1
  - *Description:* Snowbird #1.
- **2019-11-23**: Practice.
- **2019-11-24**: Practice.
- **2019-11-29**: Longer.  At dock for data collection.
- **2019-12-06**: Short, at dock.
- **2019-12-07**: Snowbird #2
  - *Description:* Snowbird #2.
- **2020-01-18**: Practice
- **2020-01-25**: STYC Iceberg
  - *Description:* Lot's of wind was predicted, but we got little.  We swapped on older sails, and the rig was super tight.  WE WERE SLOW!
- **2020-02-08**: Snowbird #4
  - *Description:* Snowbird #4.
- **2020-02-16**: 
- **2020-02-29**: Practice
- **2020-03-07**: CYC, Blakely Rocks, CSS
  - *Description:* Great day!  Good speed, both upwind and down.
- **2020-03-14**: 
- **2020-03-21**: 
- **2020-03-22**: 
- **2020-03-31**: 
- **2020-04-04**: 
- **2020-04-08**: 
- **2020-04-12**: Tune Up
  - *Description:* Quick tune up race vs. Creative.  We raced upwind from West Point to Meadow Point.  They were definitely a bit faster.
- **2020-04-16**: Tune-up with Creative
  - *Conditions:* Beautiful day. Winds were 3 quickly building to 10ish. Flat seas. Upwind to Pt. Wells buoy, raised and raced home to the hamburger.
- **2020-04-19**: Tune-up with Creative
  - *Conditions:* Beautiful day. Very light at times, almost glassy.
- **2020-04-24**: Tune-up with Creative
  - *Description:* Sailed up from Elliott Bay. Met Creative in the middle.
  - *Conditions:* Super light winds from the SSE (1-3) during tuneup.  Gave up a bit early.  After, on the way back, winds picked up to 8-10.
There should have been a strong current heading south, due to a large flood.  The interactions between southerly current and winds from the south may have been complex.
- **2020-04-26**: Return from Elliott Bay
- **2020-04-28**: Fun sail.
  - *Conditions:* Overcast.  Calm as we arrived on dock, but then it picked up to 10-15.  A bit rough.
- **2020-05-01**: 
- **2020-05-04**: Raises and Douses, part 2.
  - *Conditions:* Light winds and then building.
- **2020-05-06**: Out with Marisa
  - *Description:* Went out with Marisa to practice for cohabitation race
  - *Conditions:* 16 kits consistent.  Some sea state.
- **2020-05-08**: Practice
  - *Description:* Good day at about 13 knots.   Nice weather.  Just before high tide.  Current pushing out as we left the dock. Just Sara Paul on the boat.  We went pretty far up north doing tuneup. We needed Cunningham and back stay on. We have the rig tune for base setting  10 to 40 nots.  We tried the leward spin drop for the first time.  10 knots as we dropped. 
  - *Conditions:* Weird wind shift at the end of our excursion 
- **2020-05-09**: Cohabitate Race
  - *Description:* Great day on the water. Range from 10-15 Jordi of the day. Marisa came along with us.  We had a pretty major disappointment when we tried to drop the spinnaker using the letter box.
  - *Conditions:* Oscillating wind from the north.  
- **2020-05-11**: Practice 
  - *Description:* Goal was to try letterbox again but conditions were challenging. 
  - *Conditions:* Overcast with swirling wind from all four directions.  Low tide at -1.9, with wind from the south mostly.  
- **2020-05-15**: Practice with Beth and Holm
  - *Description:* Friday evening sunset sail.  Did some raises and douses
  - *Conditions:* Nice. Warm
- **2020-05-16**: Cohab 2 race
  - *Description:* Paul organized this race. The course was nwmn That means hamburger, West Point buoy , Meadowpoint, hamburger. It was supposed to be much longer but there was not much wind
  - *Conditions:* Very light to no wind at times. Strong current to start pushing us 1.2 knots south. Wind came from south and east and north.  It was hard to figure out when to use the spinnaker
- **2020-05-18**: 
- **2020-05-23**: Cohab 3
  - *Description:* Nwmn and we went to West Point again.  We tried hard to use current data but it was hard. Extra hard rounding at meadow poi t due to southern current. 
  - *Conditions:* Light wind from the south. 
- **2020-05-27**: 2020-05-27
- **2020-05-29**: Practice 
  - *Description:* We did  nmnmn.  Practiced windward douse twice and one Mexican.  Wind from the nor  th and current pushing south
  - *Conditions:* Nice!  Wind went up and down rapidly. 
- **2020-05-31**: Cohab #4
  - *Description:* Solid day of racing against “More Uff Da” and “Jaded”.  Victoria was out there too. Raining at points but dried up on the long leg. Course was nbwmnun. Our start was 10:26:19am. Finished at 1:12.  I think we won!  But still need to calculate the handicap. 
  - *Conditions:* Breeze started from the south as we warmed up but clocked around to the East so we raised spin at start on a port tack to Ballard buoy and stayed on spin to West Point.  Mexican at West Point and  went north on stbd. 
- **2020-06-01**: Monday race
  - *Description:* Creative Jubaliee and us made for a good race
- **2020-06-05**: Friday Eve practice 
  - *Description:* Light wind mild day.  Didn’t get very far but worked on job car settings 
  - *Conditions:* Warm. Light wind.  High tide
- **2020-06-06**: Cohab #5
  - *Description:* Course nmwnubn.  Our star was 10:29:26. 
  - *Conditions:* Oscillating wind with two storm clouds passing over Bainbrige. Puffs and lulls.  Wind varied from 160 to 230.  Numbers in 200s were coincident with storms.  We tried to use the drllenbaugh approach calling out persistent vs oscillating times.   We still got beat. Not sure where but near meadow point on way back was more clear 
- **2020-06-08**: TYT Monday 
  - *Description:* Rough shifting wind from south with current churn.  Jubalie, Liftoff’, Creative, and us among the J105s. Many other boats too.  We. Beat jubilee and creative but not left off. They were a couple mins ahread. Eclipse was in the mix too.  Gusto got a new phrf and maybe not flying.  Our start was 6;35. Finish 7:55
  - *Conditions:* Rolling and rough ride!  Rather fun in retrospect but a bit scary at times. 
- **2020-06-10**: Sunset Sail
  - *Description:* Sailed to Elliot Bay to look For a good race route.   Wind picked up on way back so we floated out spin at sunset.  Just beautiful!
  - *Conditions:* Gentle south west wind
- **2020-06-13**: Cohab#6 
  - *Description:* Crazy wind shifty day.  Started south to meadow point on spin. West Point.  Hamburger. At most to meadow point again on spin when wind shifted north and we had to drop. Almost hoisted spin again 10 mins later just to decide to wait again.  We got out ahead of More Jubalee and Lift Off on second leg and they never caught us. Only Valkyrie caught us but not by much!  Then big storm hit and we called it early. Fun day!
  - *Conditions:* Wind oscillating 160-230. Then flipped 330-012
- **2020-06-15**: Cohab#6 
  - *Description:* Crazy wind shifty day.  Started south to meadow point on spin. West Point.  Hamburger. At most to meadow point again on spin when wind shifted north and we had to drop. Almost hoisted spin again 10 mins later just to decide to wait again.  We got out ahead of More Jubalee and Lift Off on second leg and they never caught us. Only Valkyrie caught us but not by much!  Then big storm hit and we called it early. Fun day!
  - *Conditions:* Wind oscillating 160-230. Then flipped 330-012
- **2020-06-15**: TYT Monday 
  - *Description:* Windy and rougher conditions than usual.  We lost to Creative mostly because our rounding were imperfect.  They also went lower and at least as fast down wind.  
  - *Conditions:* Lots of puffs.  We would see 11 knots on gauge but only going 4.5.  So I’d have to foot off to get speed but then the heel was so big the rail was buried. Downwind was fun!  
- **2020-06-16**: TYT Monday 
  - *Description:* Windy and rougher conditions than usual.  We lost to Creative mostly because our rounding were imperfect.  They also went lower and at least as fast down wind.  
  - *Conditions:* Lots of puffs.  We would see 11 knots on gauge but only going 4.5.  So I’d have to foot off to get speed but then the heel was so big the rail was buried. Downwind was fun!  
- **2020-06-18**: TYT Monday 
  - *Description:* Rough shifting wind from south with current churn.  Jubalie, Liftoff’, Creative, and us among the J105s. Many other boats too.  We. Beat jubilee and creative but not left off. They were a couple mins ahread. Eclipse was in the mix too.  Gusto got a new phrf and maybe not flying.  Our start was 6;35. Finish 7:55
  - *Conditions:* Rolling and rough ride!  Rather fun in retrospect but a bit scary at times. 
- **2020-06-19**: Friday practice 
  - *Description:* Nice evening sail.  80 degrees.  No competition to race so we just did laps and some experiments with drops
  - *Conditions:* Breezy 
- **2020-06-20**: Friday practice 
  - *Description:* Nice evening sail.  80 degrees.  No competition to race so we just did laps and some experiments with drops
  - *Conditions:* Breezy 
- **2020-06-20**: Cohab 7. Downtown Exploration
  - *Description:* Fresh breeze day for the race we created.  NW—spaceneedlebellharborline-NW. 
  - *Conditions:* Nice wind 10-12 up wind. 150-174 from south.    Picked up to 15-20  for the exciting spinnaker finish with 4 boats side by side.  We finished behind Lodos and needed a referee on double trouble and eclipse.  
- **2020-06-22**: TYT Monday
  - *Description:* Beautiful summer evening.  Course was NUN. Our start was 6:40.  We either had a great start or over early.  But our upwind speed was not excellent.  Biggish waves meant we had to foot off particularly on starboard.  Moose, Jubilee, Creative, Liftoff, and Gusto came out too.  
  - *Conditions:* Breezy with some sea state.  
- **2020-06-24**: TYT Monday
  - *Description:* Beautiful summer evening.  Course was NUN. Our start was 6:40.  We either had a great start or over early.  But our upwind speed was not excellent.  Biggish waves meant we had to foot off particularly on starboard.  Moose, Jubilee, Creative, Liftoff, and Gusto came out too.  
  - *Conditions:* Breezy with some sea state.  
- **2020-06-24**: Gas run
  - *Description:* Mad dash to the fuel dock to get deisel since they are now open to 5:00 
- **2020-06-27**: 2020-06-27
- **2020-06-28**: Cohab 8
  - *Description:* NBWNUN. 

Folks worked super hard to “knock the dust off” and do very well.  Might have actually helped to be shorthanded, since we needed 100% attention 100% of the time.


We focused intensely on Moose, a luxury since there were no other boats nearby (Rush was ahead the whole time!).  We marked them and beat them to U mark, and then separated 20+ boat lengths.  Congrats.


Tacks were great. Jibes were flawless.  Raise was great.  Douse worked well.  Folks did their jobs, with little practice and little review.  And no critical job got dropped.  Props to Amy for working the PIT, which has dozens of little steps (in addition to main… and we were pretty fast).


Thanks to you both for showing up early and for keeping cheerful through the near constant rain.  Almost July 4th and it was darn cold out there!  (Double thanks for Steve for getting there first and getting the boat almost completely set!)


  - *Conditions:* Weird East wind predicted.  Did not disappoint at the start.  Converged to a northerly at 6-10. 
Very light at first, but then consistent.  Consistent rain and low visibility.  Hard to find the marks.
- **2020-06-29**: TYT Monday night 
  - *Description:* Fresh breeze out there but minimal waves. Quite a nice day for sailing. 
  - *Conditions:* 10-18 knots. Liftoff, corvo, creative, moose and us out there among the 105s.  We had a good start on starboard and tacked soon to port. 
- **2020-06-30**: TYT Monday
  - *Description:* Beautiful summer evening.  Course was NUN. Our start was 6:40.  We either had a great start or over early.  But our upwind speed was not excellent.  Biggish waves meant we had to foot off particularly on starboard.  Moose, Jubilee, Creative, Liftoff, and Gusto came out too.  
  - *Conditions:* Breezy with some sea state.  
- **2020-07-01**: TYT Monday night 
  - *Description:* Fresh breeze out there but minimal waves. Quite a nice day for sailing. 
  - *Conditions:* 10-18 knots. Liftoff, corvo, creative, moose and us out there among the 105s.  We had a good start on starboard and tacked soon to port. 
- **2020-07-05**: TYT Monday
  - *Description:* Beautiful summer evening.  Course was NUN. Our start was 6:40.  We either had a great start or over early.  But our upwind speed was not excellent.  Biggish waves meant we had to foot off particularly on starboard.  Moose, Jubilee, Creative, Liftoff, and Gusto came out too.  
  - *Conditions:* Breezy with some sea state.  
- **2020-07-06**: 2020-07-06
- **2020-07-08**: CYC Wednesday #1
  - *Description:* Double handed race.  Course JMEFMEF. Pronounced J-mef-mef. That’s just our practice workout sail using the green can and meadow point.   
- **2020-07-11**: Cohab #10
  - *Description:* Course: NWNUN. Wind from 0-15 knots. Tide ebbing throughout from 8.5ft. Nice calm sea state.  10 boats out.  4 j105s. Jubilee,, Moose, and Jaded came with.  LA and Dan double handed Jaded and Erik and Patty double handed Jubi
  - *Conditions:* Wind from south. 140-200 oscillating. Some other boats out so the field is getting more crowded. 


,date,title,file,source
0,2019-10-04,Sail to Everett for Foulweather Bluff Race.,2019-10-04_16:43.pd.gz,loginfo
1,2019-10-05,Foulweather Bluff Race,2019-10-05_09:18.pd.gz,loginfo
2,2019-10-11,"Short practice, upwind tacks and downwind jibes.",2019-10-11_17:38.pd.gz,loginfo
3,2019-10-12,CYC PSSC Day 1,2019-10-12_09:45.pd.gz,loginfo
4,2019-10-18,"Short, at dock.",2019-10-18_13:51.pd.gz,loginfo
...,...,...,...,...
65,2020-07-01,TYT Monday night,,gsheet
66,2020-07-05,TYT Monday,,gsheet
67,2020-07-06,2020-07-06,2020-07-06_17:27.pd.gz,logprocess
68,2020-07-08,CYC Wednesday #1,2020-07-08_17:19.pd.gz,gsheet


In [9]:
# notebook - metadata schema... light.

# We currently do not have a schema for the metadata, it is implicit, which is dangerous.  We
# can extract fields and their types.
#
# Note, we make a simplifying assumption that we have three situations: 
# i) primitive types, ii) dicts, ii) list of dicts. 
#
# We do not have dicts containing dicts.

def schema_lite(race_data):
    "Find a lightweight schema from the existing data.  Provides a guide for future entries."
    fd = flatten_dicts("", race_data)
    return distill(fd)    

def distill(race_union):
    res = {}
    for k, v in race_union.items():
        if len(v) == 0:
            res[k] = v[0]
        elif isinstance(v[0], dict):
            # print(k, "collapsing")
            collapsed = collapse_dicts(v)
            # print(k, "distilling", collapsed)
            res[k] = distill(collapsed)
            # print(k, "done")
        else:
            res[k] = set([type(e) for e in v])
    return res

def flatten_dicts(prefix, dicts):
    "Pass over a list of dicts and extract fields, and collect all the values assined to those fields.  For fields which are list of dicts, recurse.  Combine and flatten all fields."
    res = {}
    for d in dicts:
        for k, v in d.items():
            if is_list_of_dicts(v):
                # print(k, v)
                v = flatten_dicts(k + "[]", v)
                res.update(v)
            else:
                res[prefix+k] = res.get(k, []) + [v]
    return res

def distill_types(race_union):
    "For each key in race_union return a set containing the "
    res = {}
    for k, v in race_union.items():
        res[k] = set([type(e) for e in v])
    return res

def add_key(res, key_list, val):
    if len(key_list) > 0:
        base_key = key_list[0]
        if len(key_list) == 1:
            res[base_key] = res.get(base_key, set()).union(val)
        else:
            next_dict = res.get(base_key, {})
            next_dict.update(add_key)
            if base_key in res:
                pass

### The fields in the metadata schema.

- The keys in this dict are the fields used.
   - If `foo[]bar` then field `foo` is a list of records, each containing the field `bar`.
- In both cases the value is a set of types that are encountered.


In [10]:
# notebook - extract the "schema"

schema_lite(metadata.records)

{'file': {str},
 'date': {str},
 'title': {str},
 'begin': {int},
 'end': {int},
 'source': {str},
 'description': {NoneType, str},
 'crew': {list},
 'purpose': {str},
 'conditions': {str},
 'performance': {str},
 'learnings': {str},
 'raceqs_video': {NoneType, str},
 'raceqs': {NoneType, str},
 'wind': {list},
 'shroud_name': {str},
 'shroud_tension': {list},
 'segments[]winds': {list},
 'segments[]tensions': {list},
 'segments[]port': {list},
 'segments[]stbd': {list},
 'segments[]thoughts': {str},
 'questions[]text': {str},
 'questions[]author': {str},
 'questions[]context': {str},
 'questions[]proposed_solution': {str},
 'observations': {str},
 'warnings': {str},
 'segments[]name': {str},
 'wave': {str},
 'settings': {str},
 'segments[]start': {datetime.datetime},
 'segments[]end': {datetime.datetime},
 'other': {str},
 'timestamp': {datetime.datetime},
 'shroud_tensions': {str},
 'port_pointing': {str},
 'stbd_pointing': {str},
 'fluids': {str},
 'fluid_comments': {str}}

## Reading metadata from Google Forms

We created a Google form for post race metadata entry: https://forms.gle/JENZZdSWKNuoF8icA

The advantage of this **public** form is that it is easy to enter data from any device (including mobile) at any time.  And the schema is at least "weakly" enforced.

The form provide a scheme for publishing the resulting data as a spreadsheet, which is here: https://docs.google.com/spreadsheets/d/e/2PACX-1vS5g8oeSAMk-CFP-xDi4hu9a23W-iF5SMNjap-Gd78BPWvhA1GGgpDqFkQaEUVD3zoM9Pud1fozuDn8/pub?output=csv



In [11]:
# notebook 

URL = r"https://docs.google.com/spreadsheets/d/e/2PACX-1vS5g8oeSAMk-CFP-xDi4hu9a23W-iF5SMNjap-Gd78BPWvhA1GGgpDqFkQaEUVD3zoM9Pud1fozuDn8/pub?output=csv"

raw_gsheet = pd.read_csv(URL)  # index_col=0, parse_dates=True)

for c in raw_gsheet.columns:
    print(repr(c))



'Timestamp'
'Date YYYY-MM-DD (e.g. "2020-05-10") or blank for today.'
'Title: short name for sail (e.g. SBYC Snowbird #1)'
'Description (2-3 sentences, optional)'
'Purpose'
'Winds in knots (pick best description)'
'Wave height (pick best)'
'Conditions (i.e. description)'
'Settings Summary (how were sail controls set? trim?)'
'Shrouds (short name)'
'Shroud tensions (UP, MID, LOW: comma sep: 29,10,0). Pos low is tension, neg low is circle size in cm).'
'Crew'
'Additional Crew (comma separated)'
'Performance (i.e. how did we perform vs other boats or polars).'
"Learnings (something we'd like to repeat or avoid)"
'Other (try to be structured!)'
'Port Pointing'
'Starboard Pointing'
'Gas, Water, Pump Out, Empty Bilge?'
'Comments on Fluids?'
'Warnings (needed repair, change, etc).'


In [12]:
# These are the current list of columns used in the form.  Note the column names are long and verbose,
# in Google Forms the column names are also the documentation for the form fields.

# Long name and a convenient short form.
SHORT_COLNAME_TO_LONG = {
    'date'            : 'Date YYYY-MM-DD (e.g. "2020-05-10") or blank for today.',
    'title'           : 'Title: short name for sail (e.g. SBYC Snowbird #1)', 
    'purpose'         : 'Purpose',
    'crew'            : 'Crew',
    'description'     : 'Description (2-3 sentences, optional)',
    'conditions'      : 'Conditions (i.e. description)',
    'performance'     : 'Performance (i.e. how did we perform vs other boats or polars).',
    'learnings'       : "Learnings (something we'd like to repeat or avoid)",
    'warnings'        : 'Warnings (needed repair, change, etc).',
    'wave'            : 'Wave height (pick best)',
    'wind'            : 'Winds in knots (pick best description)',
    'port_pointing'   : 'Port Pointing',
    'stbd_pointing'   : 'Starboard Pointing',
    'settings'        : 'Settings Summary (how were sail controls set? trim?)',
    'shroud_name'     : 'Shrouds (short name)',
    'shroud_tension'  : 'Shroud tensions (UP, MID, LOW: comma sep: 29,10,0). Pos low is tension, neg low is circle size in cm).',
    'other'           : 'Other (try to be structured!)',
    'additional_crew' : 'Additional Crew (comma separated)',
    'timestamp'       : 'Timestamp',
    'fluid_comments'  : 'Comments on Fluids?',
    'fluids'          : 'Gas, Water, Pump Out, Empty Bilge?',
}

LONG_COLNAME_TO_SHORT = {v:k for k, v in SHORT_COLNAME_TO_LONG.items()}

In [13]:
def read_gsheet():
    "Read the latest GSHEET.  Check that nothing bad has happened, and convert to short names."
    gs = pd.read_csv(G.GSHEET_URL)
    check_columns_changed(gs, LONG_COLNAME_TO_SHORT)
    return convert_to_short_names(gs)

def check_columns_changed(df, long_colname_map):
    """
    Its entirely possible that I will someday edit the form and then the columns will get
    out of whack.  Check to see that there are neither new fields or missing fields.
    """
    new_colnames = []
    missing_colnames = []
    for c in df.columns:
        if c not in long_colname_map:
            new_colnames.append(c)
    for c in long_colname_map:
        if c not in df.columns:
            missing_colnames.append(c)
    if len(new_colnames) == 0 and len(missing_colnames) == 0:
        G.logger.info("No missing or extra columns.")
        return True
    else:
        G.logger.warning(f"Uh Oh. New cols {new_colnames}. Missing cols {missing_colnames}.")
        return False

def convert_to_short_names(raw_metadata):
    "Assuming the columns names have not changed, convert to a short form."
    return raw_metadata.rename(LONG_COLNAME_TO_SHORT, axis='columns')

In [14]:
# notebook - read the sheet and display the rows

gsheet = read_gsheet()
gsheet

2020-07-12 23:23:53,833|INFO|check_columns_changed| No missing or extra columns.


,timestamp,date,title,description,purpose,wind,wave,conditions,settings,shroud_name,...,crew,additional_crew,performance,learnings,other,port_pointing,stbd_pointing,fluids,fluid_comments,warnings
0,5/8/2020 14:50:22,2020-04-16,Tune-up with Creative,"Upwind to Pt. Wells buoy, raised and raced hom...",tune_up,4-10,flat,Beautiful day. Winds were 3 quickly building t...,NaN,"New ""base""?",...,"Sara, Paul",NaN,Good height and speed vs. Creative on the way ...,Let the sails out for downwind: both main and ...,NaN,NaN,NaN,NaN,NaN,NaN
1,5/8/2020 9:37:40,2020-05-06,Practice,Went out with Marisa to practice for cohabitat...,practice,8-14,1-2,Great,Not quite right,Light wind,...,"Sara, Marisa, Paul",NaN,Not measurd,Departure was tricky with strong wind from nor...,NaN,NaN,NaN,NaN,NaN,NaN
2,5/8/2020 21:32:07,2020-05-08,Practice,Good day at about 13 knots. Nice weather. J...,practice,8-14,1-2,Weird wind shift at the end of our excursion,3 inches on the back stay main ion on tight Cu...,Base,...,"Sara, Paul",NaN,No other boats out there.,We can do a leward drop.,Prep for Cohabitation Race after lockdown. Lo...,NaN,NaN,NaN,NaN,NaN
3,5/9/2020 16:37:05,2020-05-09,Cohabitate Race,Great day on the water. Range from 10-15 Jordi...,race,8-14,1-2,Oscillating wind from the north.,It was trimmed out well. All adjusted the uppe...,Modified,...,"Sara, Marisa, Paul",NaN,Came in second place behind Poke and Dystroy. ...,Learn the letterbox drop. Don’t Try new thing...,Lube turnbuckles,NaN,NaN,NaN,NaN,NaN
4,5/18/2020 12:53:05,2020-05-11,Practice,Goal was to try letterbox again but conditions...,practice,12-18,1-2,Overcast with swirling wind from all four dire...,Tight,Mystery,...,"Sara, Paul",NaN,Nobody else out. Boat felt hard to control,Always check shrouds at the dock.,Try to get breast line on tight in docking.,25-27,28-30,Empty Bilge?,NaN,NaN
5,5/16/2020 23:40:15,2020-05-15,Practice with Beth and Holm,Friday evening sunset sail. Did some raises a...,practice,4-10,flat,Nice. Warm,NaN,Light wind,...,"Sara, Paul",NaN,Very similar to Gusto,Stay out for snacks at sunset.,NaN,31-33,20-24,NaN,NaN,NaN
6,5/16/2020 23:37:18,2020-05-16,Cohab 2 race,Paul organized this race. The course was nwmn ...,race,0-6,flat,Very light to no wind at times. Strong current...,Lots of twist. No back stay. No Cunningham,Extra light wind. No yet on chart.,...,"Sara, Paul",NaN,Good on down wind. Recovered from a bad start ...,If you can’t take because the wind is so light...,Engine ran well,28-30,28-30,Empty Bilge?,NaN,NaN
7,5/23/2020 17:32:46,NaN,Cohab 3,Nwmn and we went to West Point again. We trie...,race,0-6,flat,Light wind from the south.,Lots of twist. Cars at 6,Light wind,...,"Sara, Paul",NaN,Good upwind. Lost on down wind leg,Review timing to turn down on windward drop. ...,Current was very strong at meadow point. \nCle...,28-30,28-30,Empty Bilge?,Used some water for flushing,NaN
8,5/29/2020 21:18:34,NaN,Practice,We did nmnmn. Practiced windward douse twice...,practice,8-14,1-2,Nice! Wind went up and down rapidly.,Back stay on and off a bunch,Base,...,"Sara, Paul",NaN,No boats out,Pull out jib early on dowse when double at 8 b...,NaN,20-24,28-30,Empty Bilge?,Water in sink for some reason.,New tennis balls
9,5/31/2020 16:21:31,NaN,Cohab #4,Solid day of racing against “More Uff Da” and ...,race,4-10,flat,Breeze started from the south as we warmed up ...,Jib sheet in tight. Travelor up to bench. Jib...,Base light. They were set to base on Friday b...,...,"Sara, Paul",NaN,We caught up on the long upwind leg. Then they...,Know your marks. Think about angles. If we c...,Only 3 boats really in our competition so less...,25-27,20-24,"Empty Bilge?, Add Water?",Paul added 20 seconds of water to the tank. I...,NaN


In [1]:
# Next step is to convert the Google Form spreadsheet rows to race metadata entries.  The goal is to 
# keep the two "close" so that conversion is not onerous.

def gsheet_row_to_metadata(row):
    "Convert the Google Form spreadsheet rows to race metadata entries."
    res = {}
    # If both the date and the timestamp is missing then something must be wrong.
    if is_missing_value(row['date']) and is_missing_value(row['timestamp']):
        G.logger.info(f"Encountered a row with missing date and timestamp. Skipping.")
        return None
    for key, val in row.iteritems():
        if key == 'date' and is_missing_value(val):
            val = timestamp_convert(row['timestamp']).format("YYYY-MM-DD")
        if is_missing_value(val):
            continue
        elif key == 'timestamp':
            val = timestamp_convert(row['timestamp']).datetime
        elif key == 'wind':
            val = [int(s.strip()) for s in val.split("-")]
        elif key == 'crew':
            val = [s.strip() for s in val.split(",")]
        elif key == 'shroud_tension':
            val = [int(s.strip()) for s in val.split(",")]
        res[key] = val
    res['source'] = 'gsheet'
    # It is a bit easier if the fields are in the same/similar order.
    return reorder_some_keys(res, SHORT_COLNAME_TO_LONG.keys())

def reorder_some_keys(dictionary, keys):
    "Return a dictionary with the keys in the order presented in keys."
    d = copy.copy(dictionary)
    res = dict()
    # Copy the ones in keys
    for k in keys:
        if k in d:
            res[k] = d.pop(k)
    # Copy the rest.
    for k in d.keys():
        res[k] = d[k]
    return res

def is_missing_value(val):
    "Pandas replaces empty CSV entries with NaN.  Return True if encountered."
    return isinstance(val, numbers.Number) and np.isnan(val)

def timestamp_convert(val):
    "Convert the google forms timestamp to a date."
    return arrow.get(val, 'M/D/YYYY H:mm:ss', tzinfo='US/Pacific')

In [1]:
# notebook - convert from gsheet to metadata format

gsheet_records = [gsheet_row_to_metadata(row) for index, row in gsheet.iterrows()]

# Show one row.
print(save_yaml(gsheet_records[-1:]))

In [1]:
# If there are new rows in the GSHEET, then add them to metadata.

def add_gsheet_records(gsheet, metadata):
    "Find records in the gsheet which are missing from the existing metadata."
    dates = metadata.dates.copy()
    res = []
    for _, row in gsheet.iterrows():
        record = gsheet_row_to_metadata(row)
        if record is not None:
            date = record['date']
            G.logger.debug(f"Examining record {date}")
            if date not in dates:
                G.logger.info(f"Found new record for {date} : {record.get('title', '')}")
                res.append(record)
            else:
                existing = dates.pop(date)
                source = existing['source']
                if source == 'byhand':
                    if metadata.timestamp < record['timestamp']:
                        G.logger.warning(f"Duplicate record. GSheet row is newer than byhand metadata: {record['timestamp']}.")
                        # Append both, we'll need to figure this out by hand
                        res.append(existing)
                        res.append(record)
                    else:
                        res.append(existing)
                elif source in ['loginfo', 'logprocess', 'gsheet']:
                    G.logger.debug(f"Merging gsheet into exiting record.")
                    # Overwrite the values in these records.
                    new_record = {**existing, **record}
                    res.append(new_record)
                else:
                    G.logger.warning(f"Found strange source: {source}.")
    return res + list(dates.values())

def update_metadata_from_gsheet():
    "Read metadata.yml and gsheet and update as needed."
    metadata = read_metadata()
    gsheet = read_gsheet()
    new_records = add_gsheet_records(gsheet, metadata)
    save_metadata(new_records)


In [1]:
# notebook 

# loginfo is a legacy location for metadata, need to pull that in...  once.

log_info = pd.read_pickle(G.LOG_INFO_PATH)
display(len(log_info))
log_info[:5]


In [1]:
def update_metadata_from_loginfo():
    "Read metadata.yml and loginfo and update as needed."
    metadata = read_metadata()
    loginfo = pd.read_pickle(G.LOG_INFO_PATH)    
    updated = merge_loginfo_records(loginfo, metadata)
    save_metadata(updated)


# Process the legacy loginfo data, this is only needed once.
def merge_loginfo_records(loginfo, metadata):
    """
    Create a metadata record for each loginfo record.  When a key already exists in
    metadata merge the info, overwriting fields in the loginfo record.
    """
    dates = metadata.dates.copy()
    rows = []
    for i, row in loginfo.iterrows():
        adt = datetime_from_log_filename(row.file)
        date_string = date_from_datetime(adt)
        record = {}
        # Munge loginfo data into "metadata" schema.
        record['file'] = row.file
        record['date'] = date_string
        record['title'], record['description'] = loginfo_title(row)
        record['begin'] = row.begin
        record['end'] = row.end
        record['source'] = 'loginfo'
        # Overwrite with the existing record... if it exists.
        if date_string in dates:
            record.update(dates.pop(date_string))
        rows.append(record)
    return rows + list(dates.values())

def loginfo_title(row):
    "Create a title and description from row record."
    if len(row.race) > 0:
        return row.race, row.description
    else:
        return row.description, ''


In [1]:
# Finally, during upload we should ensure that there is a default and empty metadata record 
# for each race.

def add_missing_metadata():
    """
    Working backward from the full list of pandas datafiles, ensure there is a default
    entry in the metadata file for each.
    """
    metadata = read_metadata()
    dates = metadata.dates.copy()
    new_dates = {}
    pfiles = p.pandas_files()
    G.logger.info(f"Found {len(pfiles)} pandas files.")
    for f in sorted(pfiles):
        adt = datetime_from_log_filename(f)
        date = date_from_datetime(adt)
        G.logger.debug(f"Examining {date} : {f}")
        # Default metadata... basically empty
        record = dict(file=f, date=date, title=date, begin=0, end=-1, source='logprocess')
        if date not in dates:
            G.logger.info(f"Found missing entry for {date} : {f}.")
        if date in new_dates:
            G.logger.warning(f"Two files for {date}. Watch out. Skipping.")
        else:
            existing = dates.pop(date, {})
            record.update(existing)
            new_dates[date] = record
    all_records = list(new_dates.values()) + list(dates.values())
    G.logger.info(f"Outputting {len(all_records)} records.")
    save_metadata(all_records)


def datetime_from_log_filename(filename, time_zone='US/Pacific'):
    "Extracts the datetime from a log filename."
    dt_string = re.sub(r".gz$", "", filename)   # Could be compressed
    dt_string = re.sub(r".pd$", "", dt_string)  # Standard .pd
    return arrow.get(dt_string, "YYYY-MM-DD_HH:mm", tzinfo=time_zone)
    

def date_from_datetime(adt):
    return adt.format("YYYY-MM-DD")

In [1]:
def update_race(updated_race_record):
    "Replace the race record, by date."
    date = updated_race_record['date']
    md = read_metadata()
    if date not in md.dates:
        raise Exception(f"Warning, {date} could not be found in the race logs.")
    md.dates[date] = updated_race_record
    save_metadata(list(md.dates.values()))

In [1]:
# notebook - 

if False:
    update_metadata_from_loginfo()
    add_missing_metadata()
    update_metadata_from_gsheet()